In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import torchvision
from torchvision import datasets, models, transforms

In [3]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from session import Session, TrainingSchedule
from LR_Schedule.cos_anneal import CosAnneal
from LR_Schedule.lr_find import lr_find
from callbacks import Validator
from callbacks import OneHotAccuracy
import Datasets.ImageClassifierData as ImageClassifierData

In [5]:
torch.cuda.is_available(), torch.cuda.get_device_name(0), torch.backends.cudnn.enabled

(True, 'GeForce GTX 960M', True)

In [6]:
#DATA_PATH = Path('data/hymenoptera_data')
DATA_PATH = Path('data/dogscats')

In [7]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data = ImageClassifierData.from_paths(DATA_PATH, 16, data_transforms)

In [8]:
print(next(iter(data['train'])))

[
( 0 , 0 ,.,.) = 
  0.9988  1.0673  0.7077  ...   0.8104  0.3481  0.5364
  1.1015  1.0844  0.6734  ...   0.9303  0.3823  0.7248
  1.1700  1.0331  0.6906  ...   0.9646  0.4508  0.8618
           ...             ⋱             ...          
 -0.9363 -0.7822 -0.7993  ...   1.1358  1.1700  0.9988
 -0.9363 -0.7650 -0.7308  ...   1.2043  1.1700  0.9303
 -0.8678 -0.8164 -0.7479  ...   1.1358  1.1358  1.0159

( 0 , 1 ,.,.) = 
  1.4307  1.5182  1.1856  ...   1.2731  0.8004  0.9930
  1.5357  1.5182  1.1506  ...   1.3431  0.8179  1.1856
  1.6057  1.4832  1.1681  ...   1.3782  0.8704  1.2906
           ...             ⋱             ...          
 -0.2850 -0.1275 -0.1800  ...  -1.7031 -1.5630 -1.5980
 -0.2850 -0.0749 -0.1099  ...  -1.7031 -1.6155 -1.6681
 -0.2675 -0.1275 -0.1275  ...  -1.7556 -1.5980 -1.4930

( 0 , 2 ,.,.) = 
  1.5942  1.6988  1.4722  ...   1.4722  1.0017  1.2108
  1.6988  1.7163  1.4200  ...   1.5420  1.0191  1.3677
  1.7685  1.6640  1.4548  ...   1.5594  1.0539  1.4897
          

In [9]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    figure, ax = plt.subplots()
    ax.imshow(inp)


'''
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])
'''

"\n# Get a batch of training data\ninputs, classes = next(iter(dataloaders['train']))\n\n# Make a grid from batch\nout = torchvision.utils.make_grid(inputs)\n\nimshow(out, title=[class_names[x] for x in classes])\n"

In [10]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
#model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft.fc = nn.Sequential(
    nn.BatchNorm1d(num_features=num_ftrs),
    nn.Dropout(p=.5),
    nn.Linear(num_ftrs, 2),
    nn.LogSoftmax(dim=1)
)

In [11]:
model_ft

ResNet(
  (conv1): Conv2d (3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNo

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=1e-3)

In [13]:
sess = Session(model_ft, criterion, optimizer)
sess.freeze()

In [14]:
lr_scheduler = CosAnneal(len(data['train']), T_mult=2)
validator = Validator(data['valid'], OneHotAccuracy())

In [15]:
schedule = TrainingSchedule(data['train'], callbacks=[lr_scheduler, validator])

In [16]:
lr_find(sess, data['train'], start_lr=1e-5)

TypeError: on_batch_end() missing 2 required positional arguments: 'output' and 'label'

In [ ]:
sess.set_lr(1e-2)

In [ ]:
sess.train(schedule, 3)

In [ ]:
sess.unfreeze()

In [ ]:
lr_find(sess, data['train'], start_lr=1e-5)

In [ ]:
sess.set_lr(5e-3)

In [ ]:
sess.train(schedule, 3)

In [ ]:
lr_scheduler.plot()